In [11]:
#import lightgbm as lgb
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import make_scorer
import optuna
import optuna.integration.lightgbm as lgb
from ...utils import ganancia_total_eval_nativa

ImportError: attempted relative import with no known parent package

In [5]:
X_train = np.load('../data_proceso/X_train.npy',allow_pickle=True)
y_train = np.load('../data_proceso/y_train.npy')
feature_names = list(np.load('../data_proceso/feature_names.npy'))

In [ ]:

def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [5]:
def objective(trial):    
    param = {
        'objective': 'l1',
        'metric': 'l1',
        'verbosity': -1,
        'boosting_type': 'gbdt',        
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 256),
        'seed': 1979
    }
       
    # insert code for getting X and y ready 
    dtrain = lgb.Dataset(X_train, label=y_train)

    lcv = lgb.cv(
        param, 
        dtrain, 
        verbose_eval=False)
                
    return lcv['l1-mean'][-1]+lcv['l1-stdv'][-1]

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

In [22]:
param = {
    'objective': 'binary',
    'metric': 'ganancia',
    'verbosity': -1,
    'boosting_type': 'gbdt'
}

dtrain = lgb.Dataset(X_train, label=y_train,feature_name=feature_names)

tuner = lgb.LightGBMTunerCV(param,dtrain,time_budget=180,feval=ganancia_total_eval_nativa)

[I 2023-02-10 04:17:40,468] A new study created in memory with name: no-name-c50585a6-7025-497b-9080-a95f0e11ddcd


In [23]:
tuner.run()













[I 2023-02-10 04:23:19,703] Trial 0 finished with value: 40608400.0 and parameters: {'feature_fraction': 1.0}. Best is trial 0 with value: 40608400.0.
feature_fraction, val_score: 40608400.000000:  14%|#4        | 1/7 [05:37<33:42, 337.05s/it]



  0%|          | 0/20 [00:00<?, ?it/s]



  0%|          | 0/10 [00:00<?, ?it/s]



  0%|          | 0/3 [00:00<?, ?it/s]



  0%|          | 0/20 [00:00<?, ?it/s]



  0%|          | 0/5 [00:00<?, ?it/s]


In [45]:
dir(tuner.study)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ask',
 '_directions',
 '_is_multi_objective',
 '_log_completed_trial',
 '_pop_waiting_trial_id',
 '_should_skip_enqueue',
 '_stop_flag',
 '_storage',
 '_study_id',
 '_tell',
 '_thread_local',
 'add_trial',
 'add_trials',
 'ask',
 'best_params',
 'best_trial',
 'best_trials',
 'best_value',
 'direction',
 'directions',
 'enqueue_trial',
 'get_trials',
 'optimize',
 'pruner',
 'sampler',
 'set_system_attr',
 'set_user_attr',
 'stop',
 'study_name',
 'system_attrs',
 'tell',
 'trials',
 'trials_dataframe',
 'user_attrs']

In [47]:
tuner.study.ask

<bound method Study.ask of <optuna.study.study.Study object at 0x7f1d6e3cfa00>>